In [4]:
!pip install sagemaker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.4/919.4 kB 2.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached cloudpickle-2.2.1-py3-none-any.whl (25 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Obtaining dependency information for protobuf<5.0,>=3.12 from https://files.pythonhosted.org/packages/e6/db/7b2edc72807d45d72f9db42f3eb86ddaf37f9e55d923159b1dbfc9d835bc/protobuf-4.25.1-cp37-abi3-macosx_10_9_universal2.whl.metadata
  Using cached smdebug_rulesconfig-1.0.1-py2.py3-none-any.whl (20 kB)
  Obtaining dependency information for importlib-metadata<7.0,>=1.4.0 from https://files.pythonhosted.org/packages/cc/37/db7ba97e676af155f5fcb1a35466f446eadc9104e25b83366e8088c9c926/importlib_metadata-6.8.0-py3-none-any.whl.metadata
  Obtaining dependency information for pathos from https://files.pythonhosted.org/packages/d8/08/ac94fa6f9eefe32963b8a54e573dab0dbc0d3df24fd34924bd9ce7eab7c4/patho

In [85]:
import boto3

# Create an IAM client
iam = boto3.client('iam')

# List roles
response = iam.list_roles()

# If you want to filter for SageMaker roles, you can do it in Python
sagemaker_roles = [role for role in response['Roles'] if 'SageMaker' in role['RoleName']]

# `sagemaker_roles` now contains a list of SageMaker roles

In [2]:
sagemaker_roles[0]['RoleName']

'AmazonSageMaker-ExecutionRole-20231030T210397'

In [3]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='AmazonSageMaker-ExecutionRole-20231030T210397')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml


Couldn't call 'get_role' to get Role ARN from role name ravi_tej to get Role path.


sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::005418323977:role/service-role/AmazonSageMaker-ExecutionRole-20231030T210397
sagemaker bucket: sagemaker-ap-south-1-005418323977
sagemaker session region: ap-south-1


In [4]:
import pandas as pd
import numpy as np
import json

In [5]:
system_prompt =  '''
You are the chief editor for a leading Indian financial and business news website. You evaluate critical attributes of articles to gate keep content quality. For many attributes, you will first provide a brief analysis of less than 30 words, followed by assessment.

1. analysis_is_financial_or_business_news (short text) : <analyse if article pertains to finance/business or not. government policies directly impacting indian corporations or investors are ok, but not if aren't>
2. is_financial_or_business_news (1/0) : <1 or 0 based on previous attribute>
3. analysis_of_relevant_for_india (short text) : <analyse if article is relevant for indians. for example articles about 401k or small foreign companies won't be relevant for india. however changes to fed interest rates or nasdaq or large multinational important news will be relevant>
4. relevant_for_india (1/0) : <1 or 0 based on previous attribute>
5. analysis_of_article_validity_duration (short text) : <Analyse relevance duration: Stock fluctuations, 1 day; significant policy changes, weeks; educational content is timeless unless it refers to any tax or other regulations in which case only 30 days. International news in India has shorter lifespan. popular topics are usually not timeless; quarterly analysis is valid for a week, yearly for a couple of weeks and a much longer one for a month>
6. article_validity_duration (one of 1, 3, 7, 14, 30, -1) : <calculate number of days based on previous attribute. -1: timeless. 1: article is relevant only for that day. 3: for a couple of days. 7: for a week. 14: for a couple of weeks. 30: for a month>
7. analysis_of_popularity (short text) : <analyse likely popularity of article - if its for niche audience, moderate_popularity or should be part of breaking_news section, depending on number of people who will be impacted by the news and the scale of the event. foreign entities known in india but not very popular will be mostly niche or rarely moderately popular. articles targeted to very specific business or pratices will be niche. infotainment business and financial articles with some drama are likely to be more popular. articles with a list of rules without compelling story-telling will be for niche audience>
8. popularity (one of niche, moderately_popular, breaking_news) : <based on previous attribute>
9. analysis_of_article_type (short text) : <analyse if the article is majorly factual, is an opinion piece, analysis, educational or likely sponsored. factual articles relay events. opinion pieces have predictions either from the author or from statements without data. analysis pieces have substantial data to justify. if an article is overly zealous on certain stock and seems like an ad, then it is sponsored>
10. article_type (one of fact, opinion, analysis, educational, sponsored) : <based on previous attribute>
11. analysis_of_article_sentiment (short_text): <analyse if the sentiment of the article is bullish, bearish or NA. balanced is NA>
12. article_sentiment (one of bull, bear, NA): <based on previous attribute>
13. summary_list (text of 60 words) : <Generate increasingly concise, entity-dense summaries. Repeat the following steps 3 times. 
Step a. Identify 1-3 informative Entities (semicolon delimited) from the Article which are missing from the previously generated summary. 
Step b. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the Missing Entities. 

A Missing Entity is: 
Relevant to the main story: Specific, descriptive yet concise (5 words or fewer); 
Novel: not in the previous summary; 
Faithful: present in the Article; 

Guidelines: Don't refer to the article in third person, but summarize the contents. The first summary should be long (4-5 sentences, ~60 words) yet highly non-specific. Subsequently rewrite the previous summary to improve flow and make space for additional entities. Make space with compression and removal of uninformative phrases like 'the article discusses'. The summaries should become highly dense but easily understood without the Article. Never drop entities from the previous summary. If space cannot be made, add fewer new entities. Remember, use the exact same number of words for each summary. generate a list of 3 dictionaries whose keys are 'missing_entities' and 'denser_summary'. and put the entire list in double quotes (") for safe overall json decoding>
14. headline_suggestion (short text) : <Write a headline based on the content of the article>

your response should be a list of 2 json structures with all the 14 above keys without missing any key. It is very important that the response is directly readable with json.loads(). no preamble or postamble. respond in the exact following structure:
{"attributes":
{
"analysis_is_financial_or_business_news": "",
"is_financial_or_business_news": "",
"analysis_of_relevant_for_india": "",
"relevant_for_india": "",
"analysis_of_article_validity_duration": "",
"article_validity_duration": "",
"evaluation_of_article_validity_duration": "",
"analysis_of_popularity": "",
"popularity": "",
"analysis_of_article_type": "",
"article_type": "",
"analysis_of_article_sentiment": "",
"article_sentiment": "",
"assessment_article_type": "",
"headline_suggestion": ""},
"summaries": [{"missing_entities":"", "denser_summary": ""}, {"missing_entities":"", "denser_summary": ""}, {"missing_entities":"", "denser_summary": ""}]
}

|article|
'''

In [235]:
old_system_prompt = '''
You are the chief editor for a leading Indian financial and business news website. You evaluate critical attributes of articles to gate keep content quality. For many attributes, you will first provide a brief analysis of 15 to 30 words, followed by assessment.

1. analysis_is_financial_or_business_news (short text) : <analyse if article pertains to finance/business or not. government policies directly impacting indian corporations or investors are ok, but not if aren't>
2. is_financial_or_business_news (True or False) : <True or False based on previous attribute>
3. analysis_of_relevant_for_india (short text) : <analyse if article is relevant for indians. for example articles about 401k or small foreign companies won't be relevant for india. however changes to fed interest rates or nasdaq or large multinational important news will be relevant>
4. relevant_for_india (True or False) : <True or False based on previous attribute>
5. analysis_of_article_validity_duration (short text) : <Analyse relevance duration: Stock fluctuations, 1 day; significant policy changes, weeks; educational content is timeless unless it refers to any tax or other regulations in which case only 30 days. International news in India has shorter lifespan. popular topics are usually not timeless; quarterly analysis is valid for a week, yearly for a couple of weeks and a much longer one for a month>
6. article_validity_duration (one of 1, 3, 7, 14, 30, -1) : <calculate number of days based on previous attribute. -1: timeless. 1: article is relevant only for that day. 3: for a couple of days. 7: for a week. 14: for a couple of weeks. 30: for a month>
7. analysis_of_popularity (short text) : <analyse likely popularity of article - if its for niche audience, moderate_popularity or should be part of breaking_news section, depending on number of people who will be impacted by the news and the scale of the event. foreign entities known in india but not very popular will be mostly niche or rarely moderately popular. articles targeted to very specific business or pratices will be niche. infotainment business and financial articles with some drama are likely to be more popular. articles with a list of rules without compelling story-telling will be for niche audience>
8. popularity (one of niche, moderately_popular, breaking_news) : <based on previous attribute>
9. analysis_of_article_type (short text) : <analyse if the article is majorly factual, is an opinion piece, analysis, educational or likely sponsored. factual articles relay events. opinion pieces have predictions either from the author or from statements without data. analysis pieces have substantial data to justify. if an article is overly zealous on certain stock and seems like an ad, then it is sponsored>
10. article_type (one of fact, opinion, analysis, educational, sponsored) : <based on previous attribute>
11. analysis_of_article_sentiment (short_text): <analyse if the sentiment of the article is bullish, bearish or NA. balanced is NA>
12. article_sentiment (one of bull, bear, NA): <based on previous attribute>
13. headline_suggestion (short text) : <Write a headline based on the content of the article>
14. summary (text of 60 words) : <Generate concise, entity-dense summary. The summary should become highly dense but easily understood without the Article. Don't keep the summary too short, but limit it to no more than 60 words>

your response should be a json structure with all the 14 above keys without missing any key. It is very important that the response is directly readable with json.loads(). no preamble or postamble. respond in the exact following structure:

{
"analysis_is_financial_or_business_news": "",
"is_financial_or_business_news": "",
"analysis_of_relevant_for_india": "",
"relevant_for_india": "",
"analysis_of_article_validity_duration": "",
"article_validity_duration": "",
"analysis_of_popularity": "",
"popularity": "",
"analysis_of_article_type": "",
"article_type": "",
"analysis_of_article_sentiment": "",
"article_sentiment": "",
"headline_suggestion": "",
"summary": ""
}

|article|
'''

In [324]:
new_system_prompt = '''
You are the chief editor for a leading Indian financial and business news website. You evaluate critical attributes of articles to gate keep content quality. For many attributes, you will first provide a brief analysis of 15 to 30 words, followed by assessment.

1. analysis_is_financial_or_business_news (short text) : <analyse if article pertains to finance/business or not. government policies directly impacting indian corporations or investors are ok, but not if aren't>
2. is_financial_or_business_news (True or False) : <True or False based on previous attribute>
3. analysis_of_relevant_for_india (short text) : <analyse if article is relevant for indians. for example articles about 401k or small foreign companies won't be relevant for india. however changes to fed interest rates or nasdaq or large multinational important news will be relevant>
4. relevant_for_india (True or False) : <True or False based on previous attribute>
5. analysis_of_article_validity_duration (short text) : <Analyse relevance duration: Stock fluctuations, 1 day; significant policy changes, weeks; educational content is timeless unless it refers to any tax or other regulations in which case only 30 days. International news in India has shorter lifespan. popular topics are usually not timeless; quarterly analysis is valid for a week, yearly for a couple of weeks and a much longer one for a month>
6. article_validity_duration (one of 1, 3, 7, 14, 30, -1) : <calculate number of days based on previous attribute. -1: timeless. 1: article is relevant only for that day. 3: for a couple of days. 7: for a week. 14: for a couple of weeks. 30: for a month>
7. analysis_of_popularity (short text) : <analyse likely popularity of article - if its for niche audience, moderate_popularity or should be part of breaking_news section, depending on number of people who will be impacted by the news and the scale of the event. foreign entities known in india but not very popular will be mostly niche or rarely moderately popular. articles targeted to very specific business or pratices will be niche. infotainment business and financial articles with some drama are likely to be more popular. articles with a list of rules without compelling story-telling will be for niche audience>
8. popularity (one of niche, moderately_popular, breaking_news) : <based on previous attribute>
9. analysis_of_article_type (short text) : <analyse if the article is majorly factual, is an opinion piece, analysis, educational or likely sponsored. factual articles relay events. opinion pieces have predictions either from the author or from statements without data. analysis pieces have substantial data to justify. if an article is overly zealous on certain stock and seems like an ad, then it is sponsored>
10. article_type (one of fact, opinion, analysis, educational, sponsored) : <based on previous attribute>
11. analysis_of_article_sentiment (short_text): <analyse if the sentiment of the article is bullish, bearish or NA. balanced is NA>
12. article_sentiment (one of bull, bear, NA): <based on previous attribute>
13. headline_suggestion (short text) : <Write a headline based on the content of the article>
14. summary (text of 60 words) : <Generate concise, entity-dense summary. The summary should become highly dense but easily understood without the Article. Don't keep the summary too short, but limit it to no more than 60 words>

your response should be a json structure with all the 14 above keys without missing any key. It is very important that the response is directly readable with json.loads(). no preamble or postamble. respond in the exact following structure:

{
"analysis_is_financial_or_business_news": "",
"is_financial_or_business_news": "",
"analysis_of_relevant_for_india": "",
"relevant_for_india": "",
"analysis_of_article_validity_duration": "",
"article_validity_duration": "",
"analysis_of_popularity": "",
"popularity": "",
"analysis_of_article_type": "",
"article_type": "",
"analysis_of_article_sentiment": "",
"article_sentiment": "",
"headline_suggestion": "",
"summary": ""
}
'''

In [233]:
old_endpoint_name = 'huggingface-pytorch-tgi-inference-2023-11-29-10-43-04-737'

In [234]:
endpoint_name = 'huggingface-pytorch-tgi-inference-2023-12-01-00-40-05-774'

In [323]:
endpoint_name = 'open-hermes-25-KindleBrothWayfarer'

In [336]:
def format_article_for_prompt(article_text):
    instruction = f"<|im_start|>system\n{new_system_prompt}<|im_end|>\n"
    # not adding context as instruction ends with |actual_article|
    truncated_content = truncate_text_to_token_limit(text=article_text, encoder=tokenizer, token_limit=ARTICLE_TOKEN_LIMIT)
    context = f"### <|im_start|>user\n{truncated_content}|im_end|>\n"
    response = f"### <|im_start|>assistant\n"
    prompt = "\n\n".join([i for i in [instruction, context, response] if i is not None])
    prompt = re.sub(r'\n+','\n',prompt)
    return prompt

In [326]:
# def format_article_for_prompt(content):
#     truncated_content = truncate_text_to_token_limit(text=content, encoder=tokenizer, token_limit=ARTICLE_TOKEN_LIMIT)
#     messages = [{"role": "system", "content": new_system_prompt},
#                 {"role": "user", "content": f"|article_start|\n {truncated_content}\n|article_end|\n"}]
#     context_prompt = tokenizer.decode(tokenizer.apply_chat_template(messages, add_generation_prompt=True))
#     prompt = context_prompt
#     prompt = re.sub(r'\n+','\n',prompt)
#     return prompt

In [8]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/arcane/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/arcane/')

from src._utils import load_bertopic_model_from_hf

/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-pac

In [9]:
from src.articles.ArticleService import ArticleService

In [122]:
art = ArticleService._get_article_json_from_s3_and_api('655acf674b13023f934af2ec')

In [123]:
full_content = art['title'] + art['cleaned_text']

In [124]:
full_content

'BusinessWorld forum to focus on economic outlook, business opportunities - BusinessWorld OnlineTHE PHILIPPINE economy’s prospects as well as business opportunities and risks will be the focus of the BusinessWorld Forecast 2024 economic forum on Wednesday. The forum, which will be held at the Grand Hyatt Manila, will feature keynote presentations, panel discussions and fi reside chats on the theme “PH Rising: Keeping the Momentum.” Pavit Ramachandran, country director for the Philippines at the Asian Development Bank, will deliver a keynote speech on the Philippine economic outlook. Ndiamé Diop, country director for Brunei, Malaysia, Philippines and Thailand at the World Bank, will also give a keynote address about development imperatives in a post-pandemic world. In his keynote speech, Ragnar Gudmundsson, resident representative to the Philippines at the International Monetary Fund, will discuss Philippine fi nancial stability risks for 2024. A panel discussion on “Growing Prospects i

In [125]:
smr = sess.boto_session.client("sagemaker-runtime")

In [334]:
parameters = {
    "do_sample": True,
    "top_p": 0.9,
    "temperature": 0.5,
    "max_new_tokens": 1024,
    "repetition_penalty": 1.03,
    "stop": ["###", "</s>"],
}

In [127]:
import re

In [128]:
request = {"inputs": format_article_for_prompt(full_content), "parameters": parameters, "stream": False}

In [129]:
import time

In [130]:
start_time = time.time()
resp = smr.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(request),
    ContentType="application/json",
)
print(f'done in {time.time() - start_time}')

done in 15.922027826309204


In [131]:
k = resp['Body'].read()

json.loads(json.loads(k)[0]['generated_text'])

{'analysis_is_financial_or_business_news': 'The article discusses the upcoming BusinessWorld Forecast 2024 economic forum, focusing on the Philippine economy, business opportunities, and development.',
 'is_financial_or_business_news': True,
 'analysis_of_relevant_for_india': "The article is about the Philippines' economic outlook and business opportunities, which may not be directly relevant for India.",
 'relevant_for_india': False,
 'analysis_of_article_validity_duration': 'The article talks about a future event, hence its relevance will last until the event occurs.',
 'article_validity_duration': 14,
 'analysis_of_popularity': 'The article is about a specific event and is likely to appeal to a niche audience interested in the Philippine economy.',
 'popularity': 'niche',
 'analysis_of_article_type': 'The article is factual, presenting information about the upcoming BusinessWorld Forecast 2024.',
 'article_type': 'fact',
 'analysis_of_article_sentiment': 'The article is neutral, pro

In [43]:
article_ids = ["652cdf8e50af0e25a912311a",
"652df8f35669b40a3b5ab660",
"652df8dc5669b40a3b5ab607",
"655acf984b13023f934af2ef",
"655acf824b13023f934af2ee",
"655acf734b13023f934af2ed",
"655acf674b13023f934af2ec",
"655acf664b13023f934af2eb",
"655acf5d4b13023f934af2ea"]

In [57]:
article_ids= pd.read_csv('data-1701156717673.csv')

In [61]:
article_ids = [k[0] for k in article_ids.values]

In [132]:
article_ids = [
"651dcf2ea662d76276b7cea4",
"652cacd650af0e25a912308e",
"652caceb50af0e25a91230cc",
"652cacf250af0e25a91230f3",
"652df8eb5669b40a3b5ab638",
"652df8ee5669b40a3b5ab645",
"652df8ef5669b40a3b5ab64a",
"652df9085669b40a3b5ab6fb",
"652e31195669b40a3b5ab767",
"652ebb761e5cc42b1b1398c4"]

In [28]:
import time

In [64]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [335]:
def calculate_tokens(text, encoder):  # Placeholder for your actual token calculation function
    # Your implementation will go here.
    return len(encoder.encode(text))  # Example: counting characters as tokens

def truncate_text_to_token_limit(text,encoder, token_limit):
    # First, check if the whole text is under the token limit
    if calculate_tokens(text, encoder) <= token_limit:
        return text  # The entire text is within the limit

    def is_under_limit(index):
        # Use the provided function to calculate tokens for the substring
        return calculate_tokens(text[:index], encoder) <= token_limit

    left, right = 0, len(text)
    valid_limit = 0  # This will hold the index of the last valid token position

    # Binary search to find the token limit
    while left <= right:
        mid = (left + right) // 2  # Find the midpoint
        if is_under_limit(mid):
            # If the midpoint is under the limit, store it as a valid limit
            valid_limit = mid
            left = mid + 1  # Move the left boundary to the right
        else:
            right = mid - 1  # Move the right boundary to the left

    # Find the last space before the valid_limit to ensure we're at a word boundary
    space_index = text.rfind(' ', 0, valid_limit)
    if space_index == -1:
        # If there's no space, we've hit the start of the text
        return text[:valid_limit]  # Return up to the valid limit even if mid-word

    # Return the text up to the last word within the token limit
    return text[:space_index]

# Example usage:
text = "Your text goes here, and it could be very long."
token_limit = 100  # Replace with your actual token limit

In [76]:
s = find_token_limit(text = k, encoder=tokenizer, token_limit=1643)

In [134]:
tokenizer = AutoTokenizer.from_pretrained('teknium/OpenHermes-2.5-Mistral-7B', use_fast=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [135]:
prompt_tokens = tokenizer(system_prompt)

In [136]:
len(prompt_tokens['input_ids'])

1429

In [363]:
def get_attributes_from_llm(article_id):
    try:
        art = ArticleService._get_article_json_from_s3_and_api(article_id)
    except:
        print(article_id, "no data")
        return article_id, 'nothing'
    full_content = art['title'] + art['cleaned_text']
    full_response = {}
    full_response['content'] = full_content
    full_input = format_article_for_prompt(full_content)
    return full_input
    # truncated_input = truncate_text_to_token_limit(full_input, encoder=tokenizer, token_limit=3072)
    request = {"inputs": full_input, "parameters": parameters, "stream": False}
    try:
        resp = smr.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(request),
            ContentType="application/json",
            )
        k = resp['Body'].read()
        return k
        # try:
        #     full_response['llm_attributes'] = json.loads(k)[0]['generated_text']
        #     print(article_id, full_response['llm_attributes'])
        #     return article_id, full_response
        # except:
        #     print(article_id, k)
        #     return article_id, k
    except:
        print(article_id, 'some error')
        article_id, ''

In [247]:
art = ArticleService._get_article_json_from_s3_and_api('6555cbba4b13023f934a0d12')

In [333]:
OUTPUT_TOKEN_LIMIT = 1024
INSTRUCTION_TOKENS = len(tokenizer.encode(new_system_prompt))

BUFFER_TOKENS = 21

ARTICLE_TOKEN_LIMIT = 4096 - OUTPUT_TOKEN_LIMIT - INSTRUCTION_TOKENS - BUFFER_TOKENS

ARTICLE_TOKEN_LIMIT

1976

In [252]:
full_content = art['title'] + art['cleaned_text']
full_input = format_article_for_prompt(full_content)

In [263]:
get_attributes_from_llm('653168c51e5cc42b1b13b90b')

653168c51e5cc42b1b13b90b


In [137]:
from concurrent.futures import ThreadPoolExecutor, as_completed

In [138]:
start_time = time.time()
import boto3
import concurrent.futures
# # Your IDs and corresponding data
# request_data = {
#     'id1': 'data1',
#     'id2': 'data2',
#     # ... add more id: data pairs
# }

# Using ThreadPoolExecutor to make parallel calls
responses = {}
with ThreadPoolExecutor(max_workers=10) as executor:
    # Submit each call to the executor
    futures = [executor.submit(get_attributes_from_llm, art_id) for art_id in article_ids]
    # Collect the responses in a dictionary
    for future in as_completed(futures):
        art_id, response = future.result()
        responses[art_id] = response
#     future_to_id = {executor.submit(get_attributes_from_llm, article_id): article_id for article_id in article_ids}

#     # Collect the responses in a dictionary
#     for future in concurrent.futures.as_completed(future_to_id):
#         article_id = future_to_id[future]
#         try:
#             _, response = future.result()
#             responses[article_id] = response
#         except Exception as exc:
#             print(f'ID {article_id} generated an exception: {exc}')

# responses now contains your results
print(f'done with {len(article_ids)} in {time.time() - start_time} seconds')

done with 10 in 28.532365083694458 seconds


### Testing

In [170]:
with open('test_set_full_27_nov.json', 'r') as f:
    test_set = json.load(f)

In [175]:
test_article_ids = list(test_set.keys())

In [180]:
# def get_attributes_for_article_ids(article_ids):
#     start_time = time.time()
#     # Using ThreadPoolExecutor to make parallel calls
#     responses = {}
#     with ThreadPoolExecutor(max_workers=10) as executor:
#         # Submit each call to the executor
#         futures = [executor.submit(get_attributes_from_llm, art_id) for art_id in article_ids]
        
#         valid_attributes = {}
#         invalid_response_ids = []
#         # Collect the responses in a dictionary
#         for future in as_completed(futures):
#             art_id, response = future.result()
#             responses[art_id] = response
#             try:
#                 valid_attributes[art_id] = json.loads(response['llm_attributes'])
#             except:
#                 invalid_response_ids.append(art_id)
#     print(f'done with {len(article_ids)} in {time.time() - start_time} seconds')
#     return valid_attributes

In [347]:
def get_attributes_for_article_ids(article_ids, max_retries=0, retry_count=0):
    start_time = time.time()
    responses = {}
    valid_attributes = {}
    invalid_response_ids = []

    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(get_attributes_from_llm, art_id) for art_id in article_ids]
        print(f'completed in {time.time() - start_time} seconds')
        for future in as_completed(futures):
            art_id, response = future.result()
            if response is None:  # Timeout or other errors
                invalid_response_ids.append(art_id)
                continue

            responses[art_id] = response
            try:
                valid_attributes[art_id] = json.loads(response['llm_attributes'])
            except:
                return response
                # invalid_response_ids.append(art_id)

    print(f'Attempt {retry_count + 1}: Processed {len(article_ids)} IDs in {time.time() - start_time} seconds')

    if invalid_response_ids and retry_count < max_retries:
        print(f'Retrying for {len(invalid_response_ids)} invalid IDs')
        valid_attributes_retry = get_attributes_for_article_ids(invalid_response_ids, max_retries, retry_count + 1)
        valid_attributes.update(valid_attributes_retry)

    return valid_attributes

In [331]:
random_articles = np.random.choice(test_article_ids, 25)

In [364]:
b1 = get_attributes_from_llm('651dd41ca662d76276b7dca8')

In [365]:
b1

'<|im_start|>system\nYou are the chief editor for a leading Indian financial and business news website. You evaluate critical attributes of articles to gate keep content quality. For many attributes, you will first provide a brief analysis of 15 to 30 words, followed by assessment.\n1. analysis_is_financial_or_business_news (short text) : <analyse if article pertains to finance/business or not. government policies directly impacting indian corporations or investors are ok, but not if aren\'t>\n2. is_financial_or_business_news (True or False) : <True or False based on previous attribute>\n3. analysis_of_relevant_for_india (short text) : <analyse if article is relevant for indians. for example articles about 401k or small foreign companies won\'t be relevant for india. however changes to fed interest rates or nasdaq or large multinational important news will be relevant>\n4. relevant_for_india (True or False) : <True or False based on previous attribute>\n5. analysis_of_article_validity_

In [361]:
json.loads(b1)[0]['generated_text']

'You are the chief editor for a leading Indian financial and business news website. You evaluate critical attributes of articles to gate keep content quality. For many attributes, you will first provide a brief analysis of 15 to 30 words, followed by assessment.\n1. analysis_is_financial_or_business_news (short text) : <analyse if article pertains to finance/business or not. government policies directly impacting indian corporations or investors are ok, but not if aren\'t>\n2. is_financial_or_business_news (True or False) : <True or False based on previous attribute>\n3. analysis_of_relevant_for_india (short text) : <analyse if article is relevant for indians. for example articles about 401k or small foreign companies won\'t be relevant for india. however changes to fed interest rates or nasdaq or large multinational important news will be relevant>\n4. relevant_for_india (True or False) : <True or False based on previous attribute>\n5. analysis_of_article_validity_duration (short text

In [353]:
va_latest = get_attributes_for_article_ids(random_articles)

completed in 0.07057404518127441 seconds
651e1036a662d76276b872e4 
651def1aa662d76276b822fe 
6555cbba4b13023f934a0d12 
6555b5a54b13023f9348be03 
651e192ba662d76276b887da 
651e1036a662d76276b872e4 
653168c51e5cc42b1b13b90b no data
65316f061e5cc42b1b13eca2 
651debaaa662d76276b819f6 
651e1912a662d76276b887a0 
6555cc3a4b13023f934a68c8 
651debaaa662d76276b819f6 some error
6555b5a54b13023f9348be03 some error
6555b5a54b13023f9348be03 some error
651def1aa662d76276b822fe65316f061e5cc42b1b13eca2 some error
 some error
6555cb974b13023f9349f4ae some error
651e1036a662d76276b872e4 some error
651df04ba662d76276b825f3 some error
652ebbe61e5cc42b1b139c4f some error
6555b5d84b13023f9348c264 some error
651dd41ca662d76276b7dca8 
651dd41ca662d76276b7dca8 
65316fd51e5cc42b1b141f77651e1f32a662d76276b895fe 
65316f061e5cc42b1b13eca2 
6555cc194b13023f934a4fa1 
6555c49e4b13023f9348d93e 
6555c1124b13023f9348ce03 
6531690a1e5cc42b1b13cb19 
651e192ba662d76276b887da some error
653168c51e5cc42b1b13b90b no data
65316

TypeError: cannot unpack non-iterable NoneType object

In [271]:
va_new = get_attributes_for_article_ids(test_article_ids)

6555cbf84b13023f934a382d
653168c51e5cc42b1b13b90b
65316f521e5cc42b1b13f95e
Attempt 1: Processed 100 IDs in 240.7760512828827 seconds
Retrying for 3 invalid IDs
653168c51e5cc42b1b13b90b
6555cbf84b13023f934a382d
65316f521e5cc42b1b13f95e
Attempt 2: Processed 3 IDs in 0.6003978252410889 seconds
Retrying for 3 invalid IDs
65316f521e5cc42b1b13f95e
653168c51e5cc42b1b13b90b
6555cbf84b13023f934a382d
Attempt 3: Processed 3 IDs in 0.4265472888946533 seconds


In [278]:
va_new['651e1b59a662d76276b88cc8']

{'analysis_is_financial_or_business_news': "Article discusses India's economic policies and potential for investment by American businesses.",
 'is_financial_or_business_news': True,
 'analysis_of_relevant_for_india': "The article discusses India's economic policies, making it relevant for Indians.",
 'relevant_for_india': True,
 'analysis_of_article_validity_duration': 'The article discusses policies and potential investments, making it relevant for several weeks.',
 'article_validity_duration': 14,
 'analysis_of_popularity': 'The article discusses policies that may impact a broad audience of business and investors.',
 'popularity': 'moderately_popular',
 'analysis_of_article_type': 'The article is factual, discussing policies and potential investments.',
 'article_type': 'fact',
 'analysis_of_article_sentiment': 'The article is neutral, discussing policies and potential investments.',
 'article_sentiment': 'NA',
 'headline_suggestion': 'Finance Minister Invites US Businesses to Inves

In [282]:
va['651dd87da662d76276b7e750']

{'analysis_is_financial_or_business_news': 'The article is about a movie, not directly related to finance or business.',
 'is_financial_or_business_news': False,
 'analysis_of_relevant_for_india': "The movie is American, but its relevance to India is limited as it doesn't have any direct impact on Indian finance or business.",
 'relevant_for_india': False,
 'analysis_of_article_validity_duration': "The article is about an upcoming movie, thus it's relevant until the movie's release.",
 'article_validity_duration': 30,
 'analysis_of_popularity': "The article is about a popular movie franchise, but since it's about a new release, it's likely to be of moderate popularity.",
 'popularity': 'moderately_popular',
 'analysis_of_article_type': 'The article is mostly factual, providing details about the upcoming movie.',
 'article_type': 'fact',
 'analysis_of_article_sentiment': 'The article is neutral, providing information about a movie without any bullish or bearish sentiment.',
 'article_se

In [281]:
va_new['651dd87da662d76276b7e750']

{'analysis_is_financial_or_business_news': 'The article is about a film project, not finance or business.',
 'is_financial_or_business_news': False,
 'analysis_of_relevant_for_india': 'The article is about a Hollywood film project, not relevant to Indian finance or business.',
 'relevant_for_india': False,
 'analysis_of_article_validity_duration': "The article is about a future film project, so it's relevant until the film is released.",
 'article_validity_duration': -1,
 'analysis_of_popularity': "The article is about a well-known filmmaker's final project, likely to be of interest to a niche audience.",
 'popularity': 'niche',
 'analysis_of_article_type': 'The article is factual, reporting on a future film project.',
 'article_type': 'fact',
 'analysis_of_article_sentiment': 'The article is neutral, simply reporting on a future film project.',
 'article_sentiment': 'NA',
 'headline_suggestion': "George A. Romero's Final Film Project, 'Twilight of the Dead', Set to Begin Production",


In [286]:
messages = [{"role": "system", "content": system_prompt},
                    {"role": "user", "content": f"|article_start|\n {''}\n|article_end|\n"}]

In [290]:
endpoint_name

'huggingface-pytorch-tgi-inference-2023-12-01-00-40-05-774'

In [292]:
va_new_2 = get_attributes_for_article_ids(test_article_ids)

6555cbf84b13023f934a382d
653168c51e5cc42b1b13b90b
65316f521e5cc42b1b13f95e
Attempt 1: Processed 100 IDs in 396.51358580589294 seconds
Retrying for 4 invalid IDs
6555cbf84b13023f934a382d
653168c51e5cc42b1b13b90b
65316f521e5cc42b1b13f95e
Attempt 2: Processed 4 IDs in 23.909919023513794 seconds
Retrying for 3 invalid IDs
6555cbf84b13023f934a382d
653168c51e5cc42b1b13b90b
65316f521e5cc42b1b13f95e
Attempt 3: Processed 3 IDs in 0.6318709850311279 seconds


In [303]:
va_new_3 = get_attributes_for_article_ids(test_article_ids)

6555cbf84b13023f934a382d
653168c51e5cc42b1b13b90b
65316f521e5cc42b1b13f95e


ReadTimeoutError: Read timeout on endpoint URL: "https://runtime.sagemaker.ap-south-1.amazonaws.com/endpoints/huggingface-pytorch-tgi-inference-2023-12-01-00-40-05-774/invocations"

In [318]:
k = get_attributes_for_article_ids(['651e1a84a662d76276b88aee'])

In [320]:
k['llm_attributes']

'You are the chief editor for a leading Indian financial and business news website. You evaluate critical attributes of articles to gate keep content quality. For many attributes, you will first provide a brief analysis of 15 to 30 words, followed by assessment.\n1. analysis_is_financial_or_business_news (short text) : <analyse if article pertains to finance/business or not. government policies directly impacting indian corporations or investors are ok, but not if aren\'t>\n2. is_financial_or_business_news (True or False) : <True or False based on previous attribute>\n3. analysis_of_relevant_for_india (short text) : <analyse if article is relevant for indians. for example articles about 401k or small foreign companies won\'t be relevant for india. however changes to fed interest rates or nasdaq or large multinational important news will be relevant>\n4. relevant_for_india (True or False) : <True or False based on previous attribute>\n5. analysis_of_article_validity_duration (short text

In [293]:
va_new_2

{'651e1a84a662d76276b88aee': {'analysis_is_financial_or_business_news': 'The article discusses changes to a train service, not related to finance or business.',
  'is_financial_or_business_news': False,
  'analysis_of_relevant_for_india': 'The article is relevant for Indians as it discusses changes to a popular train service in India.',
  'relevant_for_india': True,
  'analysis_of_article_validity_duration': 'The article is about a change in a train service which will remain in effect until it is changed again.',
  'article_validity_duration': -1,
  'analysis_of_popularity': 'The article targets a specific audience - those interested in Indian trains, and is likely to be moderately popular among that group.',
  'popularity': 'moderately_popular',
  'analysis_of_article_type': 'The article is factual, providing details about changes to a train service.',
  'article_type': 'fact',
  'analysis_of_article_sentiment': 'The article is neutral, not expressing any bullish or bearish sentiment.

In [289]:
tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

'<|im_start|>system\n\nYou are the chief editor for a leading Indian financial and business news website. You evaluate critical attributes of articles to gate keep content quality. For many attributes, you will first provide a brief analysis of less than 30 words, followed by assessment.\n\n1. analysis_is_financial_or_business_news (short text) : <analyse if article pertains to finance/business or not. government policies directly impacting indian corporations or investors are ok, but not if aren\'t>\n2. is_financial_or_business_news (1/0) : <1 or 0 based on previous attribute>\n3. analysis_of_relevant_for_india (short text) : <analyse if article is relevant for indians. for example articles about 401k or small foreign companies won\'t be relevant for india. however changes to fed interest rates or nasdaq or large multinational important news will be relevant>\n4. relevant_for_india (1/0) : <1 or 0 based on previous attribute>\n5. analysis_of_article_validity_duration (short text) : <A

In [182]:
va = get_attributes_for_article_ids(test_article_ids)

Attempt 1: Processed 100 IDs in 477.2520661354065 seconds
Retrying for 10 invalid IDs
Attempt 2: Processed 10 IDs in 311.21561217308044 seconds
Retrying for 10 invalid IDs
Attempt 3: Processed 10 IDs in 314.3252320289612 seconds
Retrying for 10 invalid IDs
Attempt 4: Processed 10 IDs in 311.9431040287018 seconds


In [225]:
va['65367fe21e5cc42b1b143c7d']

{'analysis_is_financial_or_business_news': "The article discusses Reliance Industries' diversification into various sectors.",
 'is_financial_or_business_news': True,
 'analysis_of_relevant_for_india': 'The article is about Reliance, an Indian multinational corporation, and its growth in various sectors.',
 'relevant_for_india': True,
 'analysis_of_article_validity_duration': 'The article discusses long-term business strategies and growth of Reliance, making it timeless.',
 'article_validity_duration': -1,
 'analysis_of_popularity': 'Reliance Industries is a popular company in India, so the article will attract a wide audience.',
 'popularity': 'breaking_news',
 'analysis_of_article_type': "The article provides factual information about Reliance's business diversification.",
 'article_type': 'fact',
 'analysis_of_article_sentiment': "The article presents a bullish sentiment towards Reliance's growth and diversification.",
 'article_sentiment': 'bull',
 'headline_suggestion': "Reliance'

In [232]:
with open('openhermes-finetuned-responses-test.json', 'w') as f:
    json.dump(va, f)

In [226]:
va.keys()

dict_keys(['6555cc534b13023f934a7aa8', '651de15fa662d76276b7fecf', '651e1a84a662d76276b88aee', '6555c5694b13023f9348dcc0', '65316faa1e5cc42b1b141024', '651e1b59a662d76276b88cc8', '6555ccc84b13023f934ad130', '6555cc194b13023f934a4fa1', '651de04fa662d76276b7fbfb', '6555c1124b13023f9348ce03', '652ebb961e5cc42b1b139924', '6555ccc94b13023f934ad203', '6555cac44b13023f9349673f', '6555cb974b13023f9349f4ae', '651debaaa662d76276b819f6', '6555cc4c4b13023f934a75cf', '651e1f32a662d76276b895fe', '6555cb9f4b13023f9349fa84', '65316f5e1e5cc42b1b13fbde', '652cdf9950af0e25a912312f', '651e000ca662d76276b84c0d', '652ebbe61e5cc42b1b139c4f', '6555c3c24b13023f9348d582', '6531691c1e5cc42b1b13d0c9', '6555cbba4b13023f934a0d12', '6555c31b4b13023f9348d38e', '651def1aa662d76276b822fe', '651dd87da662d76276b7e750', '65316d2a1e5cc42b1b13e1fc', '651e1c4ea662d76276b88f01', '6555cbf84b13023f934a382d', '651e14c4a662d76276b87d81', '65316f021e5cc42b1b13ec21', '651dd877a662d76276b7e743', '653168c51e5cc42b1b13b90b', '651dd2ce

In [196]:
[k for k in va.keys() if va[k]['popularity'] == 'breaking_news']

['6555ccc84b13023f934ad130',
 '6555cac44b13023f9349673f',
 '651e000ca662d76276b84c0d',
 '651def1aa662d76276b822fe',
 '651e1c4ea662d76276b88f01',
 '651dd2aea662d76276b7d8bb',
 '653168971e5cc42b1b13b0f9',
 '651ddef1a662d76276b7f833',
 '653168b31e5cc42b1b13b59c',
 '6555ccc44b13023f934acd28',
 '6555cb434b13023f9349bb7a',
 '651df243a662d76276b82b3e',
 '652fc87e1e5cc42b1b139ff9',
 '651e1912a662d76276b887a0',
 '65367fe21e5cc42b1b143c7d']

In [211]:
va['65367fe21e5cc42b1b143c7d']

{'analysis_is_financial_or_business_news': "The article discusses Reliance Industries' diversification into various sectors.",
 'is_financial_or_business_news': True,
 'analysis_of_relevant_for_india': 'The article is about Reliance, an Indian multinational corporation, and its growth in various sectors.',
 'relevant_for_india': True,
 'analysis_of_article_validity_duration': 'The article discusses long-term business strategies and growth of Reliance, making it timeless.',
 'article_validity_duration': -1,
 'analysis_of_popularity': 'Reliance Industries is a popular company in India, so the article will attract a wide audience.',
 'popularity': 'breaking_news',
 'analysis_of_article_type': "The article provides factual information about Reliance's business diversification.",
 'article_type': 'fact',
 'analysis_of_article_sentiment': "The article presents a bullish sentiment towards Reliance's growth and diversification.",
 'article_sentiment': 'bull',
 'headline_suggestion': "Reliance'

In [218]:
a, b = get_attributes_from_llm('652fc5811e5cc42b1b139e8f')

In [223]:
json.loads(b['llm_attributes'].strip().strip('assistant'))

{'analysis_is_financial_or_business_news': 'The article discusses the real estate market in Narela, Delhi, and provides guidance for potential investors.',
 'is_financial_or_business_news': True,
 'analysis_of_relevant_for_india': 'The article is highly relevant for Indians, particularly those interested in investing in the real estate market in Delhi.',
 'relevant_for_india': True,
 'analysis_of_article_validity_duration': 'The article provides a comprehensive guide to investing in the Narela real estate market. It is timeless unless there are changes in the market conditions.',
 'article_validity_duration': -1,
 'analysis_of_popularity': 'The article is likely to be moderately popular as it discusses the real estate market in Delhi, which is of interest to many Indians.',
 'popularity': 'moderately_popular',
 'analysis_of_article_type': 'The article is an informative guide to investing in the Narela real estate market, providing factual information and advice.',
 'article_type': 'fac

In [185]:
list(va.keys())

['6555cc534b13023f934a7aa8',
 '651de15fa662d76276b7fecf',
 '651e1a84a662d76276b88aee',
 '6555c5694b13023f9348dcc0',
 '65316faa1e5cc42b1b141024',
 '651e1b59a662d76276b88cc8',
 '6555ccc84b13023f934ad130',
 '6555cc194b13023f934a4fa1',
 '651de04fa662d76276b7fbfb',
 '6555c1124b13023f9348ce03',
 '652ebb961e5cc42b1b139924',
 '6555ccc94b13023f934ad203',
 '6555cac44b13023f9349673f',
 '6555cb974b13023f9349f4ae',
 '651debaaa662d76276b819f6',
 '6555cc4c4b13023f934a75cf',
 '651e1f32a662d76276b895fe',
 '6555cb9f4b13023f9349fa84',
 '65316f5e1e5cc42b1b13fbde',
 '652cdf9950af0e25a912312f',
 '651e000ca662d76276b84c0d',
 '652ebbe61e5cc42b1b139c4f',
 '6555c3c24b13023f9348d582',
 '6531691c1e5cc42b1b13d0c9',
 '6555cbba4b13023f934a0d12',
 '6555c31b4b13023f9348d38e',
 '651def1aa662d76276b822fe',
 '651dd87da662d76276b7e750',
 '65316d2a1e5cc42b1b13e1fc',
 '651e1c4ea662d76276b88f01',
 '6555cbf84b13023f934a382d',
 '651e14c4a662d76276b87d81',
 '65316f021e5cc42b1b13ec21',
 '651dd877a662d76276b7e743',
 '653168c51e5c

In [215]:
set(test_article_ids) - set(va.keys())

{'651df8e7a662d76276b83a76',
 '652fc5811e5cc42b1b139e8f',
 '652fd13b1e5cc42b1b13a41b',
 '652fd3521e5cc42b1b13a524',
 '65316f961e5cc42b1b1409ef',
 '653680151e5cc42b1b143f8e',
 '6555b5d84b13023f9348c264',
 '6555ca884b13023f9349438c',
 '6555ca8b4b13023f934944d1',
 '6555cc3a4b13023f934a68c8'}